In [8]:
import os
import re
import argparse
import pickle

import pandas as pd
import torch

In [9]:
movies = []
with open('./dataset/movielens/movies.dat', encoding='latin1') as f:
    for l in f:
        id_, title, genres = l.strip().split('::')
        genres_set = set(genres.split('|'))

        # extract year
        assert re.match(r'.*\([0-9]{4}\)$', title)
        year = title[-5:-1]
        title = title[:-6].strip()

        data = {'movie_id': int(id_), 'title': title, 'year': year, 'genre': genres.split("|")}
        for g in genres_set:
            data[g] = True
        movies.append(data)
movies = pd.DataFrame(movies).astype({'year': 'int'})

In [10]:
movies

,movie_id,title,year,genre,Comedy,Children's,Animation,Adventure,Fantasy,Romance,...,Action,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story,1995,"[Animation, Children's, Comedy]",True,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,1995,"[Adventure, Children's, Fantasy]",NaN,True,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,1995,"[Comedy, Romance]",True,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,1995,"[Comedy, Drama]",True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II,1995,[Comedy],True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents,2000,[Comedy],True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3879,3949,Requiem for a Dream,2000,[Drama],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3880,3950,Tigerland,2000,[Drama],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3881,3951,Two Family House,2000,[Drama],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
ratings = []
with open('./dataset/movielens/ratings.dat', encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
            })
ratings = pd.DataFrame(ratings)

In [12]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [13]:
merged_ratings = pd.merge(ratings, movies, on=['movie_id'])
merged_ratings = merged_ratings[['movie_id', 'rating', 'genre']]
merged_ratings = merged_ratings.explode('genre')
genres = pd.DataFrame(merged_ratings['genre'].unique()).reset_index()
genres.columns = ['genre_id', 'genre']
merged_ratings = pd.merge(merged_ratings, genres, on='genre')
distinct_movies_in_ratings = merged_ratings['movie_id'].unique()
movies = movies[movies['movie_id'].isin(distinct_movies_in_ratings)]
genres = pd.DataFrame(genres).astype({'genre_id': 'category'})

In [14]:
movies

,movie_id,title,year,genre,Comedy,Children's,Animation,Adventure,Fantasy,Romance,...,Action,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story,1995,"[Animation, Children's, Comedy]",True,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,1995,"[Adventure, Children's, Fantasy]",NaN,True,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,1995,"[Comedy, Romance]",True,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,1995,"[Comedy, Drama]",True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II,1995,[Comedy],True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents,2000,[Comedy],True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3879,3949,Requiem for a Dream,2000,[Drama],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3880,3950,Tigerland,2000,[Drama],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3881,3951,Two Family House,2000,[Drama],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [16]:
genres

,genre_id,genre
0,0,Drama
1,1,Animation
2,2,Children's
3,3,Musical
4,4,Romance
5,5,Comedy
6,6,Action
7,7,Adventure
8,8,Fantasy
9,9,Sci-Fi


In [17]:
merged_ratings

,movie_id,rating,genre,genre_id
0,1193,5,Drama,0
1,1193,5,Drama,0
2,1193,4,Drama,0
3,1193,4,Drama,0
4,1193,5,Drama,0
...,...,...,...,...
2101810,404,5,Documentary,17
2101811,404,3,Documentary,17
2101812,2198,3,Documentary,17
2101813,2198,5,Documentary,17


In [18]:
movies['year']

0       1995
1       1995
2       1995
3       1995
4       1995
        ... 
3878    2000
3879    2000
3880    2000
3881    2000
3882    2000
Name: year, Length: 3706, dtype: int32

add_entities(entity_table, primary_key, name): 노드 데이터를 추가합니다. entity_table은 노드 데이터가 있는 데이터프레임이며, primary_key는 노드의 고유 식별자(primary key) 열 이름입니다. name은 노드 유형을 나타내는 문자열입니다.

add_binary_relations(relation_table, source_key, destination_key, name): 엣지 데이터를 추가합니다. relation_table은 엣지 데이터가 있는 데이터프레임이며, source_key와 destination_key는 각각 엣지의 출발 노드 및 도착 노드를 식별하는 열 이름입니다. name은 엣지 유형을 나타내는 문자열입니다.

build(): 추가한 노드 데이터와 엣지 데이터를 기반으로 heterogeneous graph를 생성합니다. 생성된 graph를 반환합니다.

In [27]:
from multisage.builder import PandasGraphBuilder

# 그래프의 기본적인 노드와 엣지만 생성
graph_builder = PandasGraphBuilder()
graph_builder.add_entities(genres, 'genre_id', 'genre')
graph_builder.add_entities(movies, 'movie_id', 'movie')
graph_builder.add_binary_relations(merged_ratings, 'genre_id', 'movie_id', 'define')
graph_builder.add_binary_relations(merged_ratings, 'movie_id', 'genre_id', 'define-by')
g = graph_builder.build()

In [29]:
g

Graph(num_nodes={'genre': 18, 'movie': 3706},
      num_edges={('genre', 'define', 'movie'): 2101815, ('movie', 'define-by', 'genre'): 2101815},
      metagraph=[('genre', 'movie', 'define'), ('movie', 'genre', 'define-by')])

In [31]:
# 그래프 구조 확인

print("Number of nodes per type:")
print(g.number_of_nodes())
print("Number of edges per type:")
print(g.number_of_edges())
print("Node types:")
print(g.ntypes)
print("Edge types:")
print(g.etypes)

# 노드 데이터 확인
for ntype in g.ntypes:
    print(f"Node type: {ntype}")
    print(g.nodes[ntype].data)

# 엣지 데이터 확인
for stype, etype, dtype in g.canonical_etypes:
    print(f"Edge type: {etype}")
    print(g.edges[etype].data)

# 그래프 구조 확인
print("Graph structure:")
print(g)

Number of nodes per type:
3724
Number of edges per type:
4203630
Node types:
['genre', 'movie']
Edge types:
['define', 'define-by']
Node type: genre
{}
Node type: movie
{}
Edge type: define
{}
Edge type: define-by
{}
Graph structure:
Graph(num_nodes={'genre': 18, 'movie': 3706},
      num_edges={('genre', 'define', 'movie'): 2101815, ('movie', 'define-by', 'genre'): 2101815},
      metagraph=[('genre', 'movie', 'define'), ('movie', 'genre', 'define-by')])


주어진 코드는 DGL 그래프 g에 노드와 엣지 데이터를 추가하는 과정입니다. 주어진 코드에서는 다음과 같은 작업이 수행됩니다:

'genre' 노드 유형에 'id'라는 속성을 추가하고, 이 속성에 genres['genre_id'].cat.codes.values를 LongTensor 형태로 할당합니다.

movies 데이터프레임의 'year' 열을 'category'로 변환하고, genre_columns에서 해당 열을 제외한 모든 열을 bool 타입으로 채우고, 이를 movies 데이터프레임에 다시 할당합니다.

'movie' 노드 유형에 'year'이라는 속성을 추가하고, 이 속성에 movies['year'].cat.codes.values를 LongTensor 형태로 할당합니다.

'movie' 노드 유형에 'genre'이라는 속성을 추가하고, 이 속성에 movies[genre_columns].values를 FloatTensor 형태로 할당합니다.

'define' 엣지 유형에 'rating'이라는 속성을 추가하고, 이 속성에 merged_ratings['rating'].values를 LongTensor 형태로 할당합니다.

'define-by' 엣지 유형에 'rating'이라는 속성을 추가하고, 이 속성에 merged_ratings['rating'].values를 LongTensor 형태로 할당합니다.

위의 작업들을 통해 g 그래프의 노드와 엣지 데이터가 각각 설정되고, 이제 해당 그래프를 사용하여 그래프 학습이나 분석을 수행할 수 있습니다. 이러한 노드와 엣지 속성들은 그래프 신경망(GNN) 모델의 입력이나 그래프 관련 기타 작업에 사용될 수 있습니다.

In [32]:
# 그래프 데이터 변형
g.nodes['genre'].data['id'] = torch.LongTensor(genres['genre_id'].cat.codes.values)
movies = pd.DataFrame(movies).astype({'year': 'category'})
genre_columns = movies.columns.drop(['movie_id', 'title', 'year', 'genre'])
movies[genre_columns] = movies[genre_columns].fillna(False).astype('bool')
g.nodes['movie'].data['year'] = torch.LongTensor(movies['year'].cat.codes.values)
g.nodes['movie'].data['genre'] = torch.FloatTensor(movies[genre_columns].values)
g.edges['define'].data['rating'] = torch.LongTensor(merged_ratings['rating'].values)
g.edges['define-by'].data['rating'] = torch.LongTensor(merged_ratings['rating'].values)

In [33]:
# 그래프 구조 확인

print("Number of nodes per type:")
print(g.number_of_nodes())
print("Number of edges per type:")
print(g.number_of_edges())
print("Node types:")
print(g.ntypes)
print("Edge types:")
print(g.etypes)

# 노드 데이터 확인
for ntype in g.ntypes:
    print(f"Node type: {ntype}")
    print(g.nodes[ntype].data)

# 엣지 데이터 확인
for stype, etype, dtype in g.canonical_etypes:
    print(f"Edge type: {etype}")
    print(g.edges[etype].data)

# 그래프 구조 확인
print("Graph structure:")
print(g)

Number of nodes per type:
3724
Number of edges per type:
4203630
Node types:
['genre', 'movie']
Edge types:
['define', 'define-by']
Node type: genre
{'id': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])}
Node type: movie
{'year': tensor([75, 75, 75,  ..., 80, 80, 80]), 'genre': tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge type: define
{'rating': tensor([5, 5, 4,  ..., 3, 5, 4])}
Edge type: define-by
{'rating': tensor([5, 5, 4,  ..., 3, 5, 4])}
Graph structure:
Graph(num_nodes={'genre': 18, 'movie': 3706},
      num_edges={('genre', 'define', 'movie'): 2101815, ('movie', 'define-by', 'genre'): 2101815},
      metagraph=[('genre', 'movie', 'define'), ('movie', 'genre', 'define-by')])


In [20]:
# 그래프 저장

import dgl

output_path = 'graph_data.dgl'
dgl.save_graphs(output_path, [g])